# Object Tracking with YOLOv3

In [1]:
import matplotlib.pyplot as plt
import cv2
import random, os
import numpy as np
from glob import glob

In [2]:
config = "../yolo_files/yolov3.cfg"
weights = "../yolo_files/yolov3.weights"
labels_path = "../yolo_files/coco.names"

with open(labels_path, 'r') as f:
    classes = f.read().strip().split('\n')

In [3]:
video_path = '../yolo_files/car_flow.mp4'

output_path = 'object_tracking_yolov3.mp4'

In [4]:
def load_yolo_model(config_path, weights_path):
    
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
    
    return net, output_layers

def detect_objects(net, frame, output_layers):
    
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    return boxes, confidences, class_ids


def draw_labels(boxes, confidences, class_ids, classes, frame):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {round(confidence, 2)}", (x, y + 30), font, 2, color, 2)

net, output_layers = load_yolo_model(config, weights)

# ----------------------------------------

cap = cv2.VideoCapture(video_path) # capture/read video

fourcc = cv2.VideoWriter_fourcc(*'XVID') # XVID => Codec type required to read the video; Good for video compression and stabilization.

out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
# 30.0 => fps(frame per second)
#  (int(cap.get(3)), int(cap.get(4))) => Those values are taken from the width and height index information of the video.

# THE CODE BLOG FOR MAKING FRAME BY FRAME OPERATIONS ON VIDEO!
while True:
    ret, frame = cap.read() # cap => "video capture"
    if not ret:
        break
    boxes, confidences, class_ids = detect_objects(net, frame, output_layers)
    draw_labels(boxes, confidences, class_ids, classes, frame)
    out.write(frame) # The processed frames are added to "out".

cap.release()
out.release()
cv2.destroyAllWindows()

from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Object Tracking with Ultranalytics(YOLOv8)

In [6]:
# !pip install ultralytics

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
# METHOD-1
# !yolo track source='../yolo_files/car_flow.mp4' save=True
# We call "yolo" and say that -track and save- the specified source.

In [9]:
# METHOD-2 BeGIN
video_path = '../yolo_files/car_flow.mp4'

output_path = 'object_tracking_yolov8.mp4'

In [10]:
model = YOLO("yolov8n.pt")

names = model.model.names

track_history = defaultdict(lambda: []) # Used to store object history

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file" # Give a warning if the video cannot be opened!

w, h, fps = ( int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS) )

result = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

# THE CODE BLOG FOR MAKING FRAME BY FRAME OPERATIONS ON VIDEO!
while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        results = model.track(frame, persist=True, verbose=False) # Make observations on frames with the model!
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist() # class infos
            track_ids = results[0].boxes.id.int().cpu().tolist() # id info
            confs = results[0].boxes.conf.float().cpu().tolist() # confidence scores

            # Annotator Init
            annotator = Annotator(frame, line_width=2) # Mark the frames with Annotator()!

            # Go through the boxes one by one and make drawings!
            for box, cls, track_id in zip(boxes, clss, track_ids):

                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))

                if len(track) > 30:
                    track.pop(0)

                # Plot tracks - Mark the tracking points!
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame) # Add each processed frame to the result created with VideoWriter()!
        if cv2.waitKey(1) & 0xFF == ord("q"): # Press the "q" key on the keyboard to exit!
            break
    else:
        break


result.release()
cap.release()
cv2.destroyAllWindows()


from google.colab import files
files.download(output_path)

100%|██████████| 6.25M/6.25M [00:00<00:00, 77.9MB/s]


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.3s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Object Counting with YOLOv8

In [11]:
# !pip install ultralytics
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

In [12]:
video_path = '../yolo_files/people.m4v'

output_path = "people_counting_yolov8.mp4"

In [15]:
region_points = [
    (359, 138), # up-left
    (359, 388), # down-left
    (409, 388), # up-right
    (409, 138) # down-right
] # A hypothetically designated security area!

In [ ]:
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))


counter = object_counter.ObjectCounter() # ultranalytics built-in method ObjectCounter() used in here!

# counter.set_args(view_img=True,
#                  reg_pts=region_points, # We gave the area we specified as a parameter
#                  classes_names=model.names,
#                  draw_tracks=True)


while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False) # Watch the model frame by frame!

    im0 = counter.start_counting(im0, tracks)
    # im0 = counter.count_objects(....)
    video_writer.write(im0)


cap.release()
video_writer.release()
cv2.destroyAllWindows()

from google.colab import files
files.download(output_path)

In [ ]:
import shutil, os
from glob import glob

In [ ]:
output_path = '../object_tracking_yolov8.mp4'
drive_path = '../outputs/'
os.makedirs(drive_path, exist_ok=True)
shutil.copy(output_path, drive_path)

In [ ]:
os.listdir(drive_path)

# Saving Outputs from Colab to Drive

In [ ]:
import shutil, os
from glob import glob

output_path = '../people_counting_yolov8.mp4'  # Path to the local file you want to save
drive_path = '/content/drive/MyDrive/Colab Notebooks/outputs/'  # target folder in Google Drive

# If the target directory does not exist, create it
os.makedirs(drive_path, exist_ok=True)
# copy the file
shutil.copy(output_path, drive_path)

os.listdir(drive_path)